Task 1 - Install Spark, load required libraries, set environment variables, initiate Spark, load file

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder


In [ ]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
files.upload()

In [ ]:
!ls


data.csv     spark-3.4.1-bin-hadoop3	  spark-3.4.1-bin-hadoop3.tgz.1
sample_data  spark-3.4.1-bin-hadoop3.tgz


In [ ]:
data = sc.read.csv("data.csv",inferSchema=True, header = True)

In [ ]:
data.printSchema()
data.describe().toPandas().transpose()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
Make,11914,None,None,Acura,Volvo
Model,11914,745.5822222222222,1490.8280590623795,1 Series,xD
Year,11914,2010.384337753903,7.5797398875957995,1990,2017
Engine Fuel Type,11911,None,None,diesel,regular unleaded
Engine HP,11845,249.38607007176023,109.19187025917194,55,1001
Engine Cylinders,11884,5.628828677213059,1.78055934824622,0,16
Transmission Type,11914,None,None,AUTOMATED_MANUAL,UNKNOWN
Driven_Wheels,11914,None,None,all wheel drive,rear wheel drive
Number of Doors,11908,3.4360933825999327,0.8813153865835529,2,4


In [ ]:
def replace(column,value):
  return when (column != value,column).otherwise(lit(None))
data =data.withColumn("Market Category", replace(col("Market Category"),"N/A"))

In [ ]:
data.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c)for c in data.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|           3742|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [ ]:
data = data.drop("Market Category")
data = data.na.drop()
print((data.count(),len(data.columns)))

(11812, 15)


In [ ]:
assembler = VectorAssembler(inputCols=['Year','Engine HP','Engine Cylinders','Number of Doors','highway MPG','city mpg','Popularity'],
                            outputCol='Attributes')

In [ ]:
regressor = RandomForestRegressor(featuresCol='Attributes',labelCol='MSRP')

In [ ]:
pipeline = Pipeline (stages = [assembler,regressor])
pipeline.write().overwrite().save("pipeline")
!ls

data.csv  sample_data		   spark-3.4.1-bin-hadoop3.tgz
pipeline  spark-3.4.1-bin-hadoop3  spark-3.4.1-bin-hadoop3.tgz.1


In [ ]:
pipelineModel = Pipeline.load("pipeline")
paramGrid = ParamGridBuilder() \
  .addGrid(regressor.numTrees,[100,500]) \
  .build()
crossval =CrossValidator(estimator=pipelineModel,
                         estimatorParamMaps = paramGrid,
                         evaluator = RegressionEvaluator(labelCol='MSRP'),
                         numFolds=3)

In [ ]:
train_data, test_data = data.randomSplit([0.8,0.2],seed=123)
cvModel = crossval.fit(train_data)

In [ ]:
bestModel = cvModel.bestModel
for x in range(len(bestModel.stages)):
  print(bestModel.stages[x])

VectorAssembler_c51ccfd9e6d5
RandomForestRegressionModel: uid=RandomForestRegressor_c5e519099576, numTrees=500, numFeatures=7


In [ ]:

pred =cvModel.transform(test_data)
pred.select('MSRP','prediction').show()

+-----+------------------+
| MSRP|        prediction|
+-----+------------------+
|28030|32335.918952696993|
|30550| 38027.70615073415|
|29350|28089.649454927403|
|27900|27471.612894461927|
|34890|27471.612894461927|
|32990|27471.612894461927|
| 2827| 5188.935338161641|
| 3000| 5188.935338161641|
| 3086| 5178.507374854834|
| 3130| 5178.507374854834|
| 3012| 5208.093057872805|
| 3622| 6314.972702620008|
|22300| 24459.20303817576|
|19400|  22565.3055989348|
| 2042| 5083.923951999667|
| 2144| 5207.606059415896|
|49440| 39611.28268570804|
|52640| 39611.28268570804|
|47440| 39818.60408840133|
|58400| 39847.95517513543|
+-----+------------------+
only showing top 20 rows



In [ ]:
eval = RegressionEvaluator(labelCol='MSRP')
rmse = eval.evaluate(pred)
mse = eval.evaluate(pred,{eval.metricName:"mse"})
mae = eval.evaluate(pred,{eval.metricName:"mae"})
r2 = eval.evaluate(pred,{eval.metricName:"r2"})

In [ ]:
print(rmse,mse,mae,r2)

36245.95534982731 1313769279.221675 9695.348739536266 0.7790546784255538
